# Tiền xử lý dữ liệu

## Import thư viện và load data

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

pd.options.display.float_format = '{:,.2f}'.format

In [40]:
df = pd.read_csv('../data/players_data.csv')
print(f'Kích thước dữ liệu: {df.shape}')
df.head(5)

Kích thước dữ liệu: (32601, 20)


,name,age_at_last_season,country_of_birth,country_of_citizenship,position,sub_position,foot,height_in_cm,current_club_name,current_club_domestic_competition_id,club_position,last_season,contract_expiration_date,market_value_in_eur,agent_name,total_goals,total_assists,total_minutes_played,total_yellow_cards,total_red_cards
0,Miroslav Klose,37.00,Poland,Germany,Attack,Centre-Forward,right,184.00,Società Sportiva Lazio S.p.A.,serie-a,8.00,2015,NaN,"1,000,000.00",ASBW Sport Marketing,12,8,2429,6,0
1,Roman Weidenfeller,37.00,Germany,Germany,Goalkeeper,Goalkeeper,left,190.00,Borussia Dortmund,bundesliga,4.00,2017,NaN,"750,000.00",Neubauer 13 GmbH,0,0,181,0,0
2,Dimitar Berbatov,34.00,Bulgaria,Bulgaria,Attack,Centre-Forward,NaN,NaN,Panthessalonikios Athlitikos Omilos Konstantin...,super-league-1,NaN,2015,NaN,"1,000,000.00",CSKA-AS-23 Ltd.,6,0,1656,0,0
3,Lúcio,34.00,Brazil,Brazil,Defender,Centre-Back,NaN,NaN,Juventus Football Club,serie-a,1.00,2012,NaN,"200,000.00",NaN,0,0,307,0,0
4,Tom Starke,36.00,East Germany (GDR),Germany,Goalkeeper,Goalkeeper,right,194.00,FC Bayern München,bundesliga,NaN,2017,NaN,"100,000.00",IFM,0,0,450,0,0


## Data cleaning
Dataset là màn hồ sơ, số liệu thống kê và giá trị của một cầu thủ tại mùa giải gần nhất. Nên loại bỏ các cầu thủ không thi đấu tại mùa giải gần nhất (số phút thi đấu = 0). Ngoài ra loại bỏ cả các cầu thủ không có giá trị chuyển nhượng và những giá trị phi lý (như chiều cao <100cm)

In [41]:
# Xử lý Target rỗng
df = df.dropna(subset=['market_value_in_eur'])

# Xử lý cầu thủ không thi đấu
df = df[df['total_minutes_played'] > 0]

# Xóa chiều cao vô lý 
df = df[df['height_in_cm'] >= 150]

# Xóa những cầu thủ có hợp đống kết thúc trước mùa giải gần nhất thi đấu
df['contract_expiration_date'] = pd.to_datetime(df['contract_expiration_date'], errors='coerce')
contract_year = df['contract_expiration_date'].dt.year
df = df[
    (contract_year >= df['last_season']) | 
    (pd.isna(contract_year))
]

print(f"Dữ liệu sau khi làm sạch: {df.shape}")

Dữ liệu sau khi làm sạch: (21422, 20)


## Xử lý missing values

In [42]:
# --- 1. Cột dạng Số (Numerical) ---
# Điền missing bằng Median (Trung vị) để tránh ảnh hưởng bởi outlier
num_cols = df.select_dtypes(include=[np.number]).columns
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

# --- 2. Cột dạng Phân loại (Categorical) ---
# Cột 'foot': Điền bằng giá trị phổ biến nhất (Mode)
if df['foot'].isnull().sum() > 0:
    df['foot'] = df['foot'].fillna(df['foot'].mode()[0])

# Cột 'sub_position': Điền logic dựa theo 'position' cha
# Tạo từ điển map: {Position -> Mode của Sub-position đó}
pos_map = df.groupby('position')['sub_position'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').to_dict()

# Áp dụng map vào những dòng bị thiếu sub_position
df['sub_position'] = df.apply(
    lambda row: pos_map.get(row['position'], 'Unknown') if pd.isna(row['sub_position']) else row['sub_position'],
    axis=1
)

df['country_of_citizenship'] = df['country_of_citizenship'].fillna(df['country_of_birth'])
df['country_of_citizenship'] = df['country_of_citizenship'].fillna('Unknown')

# Các cột categorical còn lại: Điền "Unknown"
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna('Unknown')

print("Đã xử lý xong Missing Values.")

Đã xử lý xong Missing Values.


### Feature engineering
Tạo ra những cột mới có ý nghĩa cho mô hình. Bao gồm có người đại diện không. Số bàn thắng + kiến tạo mỗi 90 phút. Nếu cầu thủ đó ra sân it hơn 90 phút, số G+A/90 sẽ được tính tính bằng đúng số bàn thắng và số kiến tạo của cầu thủ đó

In [43]:
import numpy as np

# 1. Has Agent
df['has_agent'] = df['agent_name'].apply(lambda x: 0 if x == 'Unknown' else 1)

# 2. Goals + Assists per 90 min (Logic xử lý cầu thủ đá ít < 90p)
df['ga_per90min'] = np.where(
    df['total_minutes_played'] < 90,
    df['total_goals'] + df['total_assists'], # Lấy giá trị tuyệt đối nếu đá ít
    (df['total_goals'] + df['total_assists']) / df['total_minutes_played'] * 90
)

# 3. Thời hạn hợp đồng (Logic mới: Năm hết hạn - Mùa giải trước)
df['contract_expiration_date'] = pd.to_datetime(df['contract_expiration_date'], errors='coerce')
df['contract_years_remaining'] = df['contract_expiration_date'].dt.year - df['last_season']
median_years = df['contract_years_remaining'].median()
df['contract_years_remaining'] = df['contract_years_remaining'].fillna(median_years)

df['contract_years_remaining'] = df['contract_years_remaining'].astype(int)

print("Đã tạo xong các Feature (Contract Years = Expiry Year - Last Season).")
print(df[['last_season', 'contract_expiration_date', 'contract_years_remaining']].head())

Đã tạo xong các Feature (Contract Years = Expiry Year - Last Season).
   last_season contract_expiration_date  contract_years_remaining
0         2015                      NaT                         3
1         2017                      NaT                         3
4         2017                      NaT                         3
7         2015                      NaT                         3
9         2015               2023-12-31                         8


## Lưu dữ liệu để sử dụng cho trả lời câu hỏi

In [44]:
df.to_csv('../data/processed_players_data.csv', index=False)